In [1]:
import pandas as pd
import re
from collections import OrderedDict

In [2]:
data=pd.read_csv("IR_CSV.csv")
stop_words=pd.read_csv("stop_words.csv")
punctuations=pd.read_csv("punctuations.csv")
mokasar=pd.read_csv("mokasar.csv")
harekat=pd.read_csv("harekat.csv")
verbs=pd.read_csv("verbs.csv")

# clean data

In [3]:
data = data[data['content'].notnull()]

# tokenize

In [4]:
%%time
def tokenize(x):
    return x.split()
tokens=[]
for doc in data["content"]:
    doc=re.sub('http://\S+|https://\S+',"",doc)
    doc=doc.replace("انتهای پیام","")
    tokens.append(set(doc.split()))


Wall time: 1.11 s


# merge

In [5]:
def merge(list1,list2):
    i=j=0
    m=[]
    while(i<len(list1) and j<len(list2)):
        
        if list1[i]<list2[j]:
            m.append(list1[i])
            i+=1
        elif list1[i]==list2[j]:
            m.append(list1[i])
            i+=1
            j+=1
        else:
            m.append(list2[j])
            j+=1
    while(i<len(list1)):
        m.append(list1[i])
        i+=1
    while(j<len(list2)):
        m.append(list2[j])
        j+=1
    return m

# یکسان سازی

# اعداد و علائم

In [6]:
chars_to_remove=''.join(punctuations["punctuations"].values.tolist()).replace(" ","")
chars_to_remove+="٠١٢٣٤٥٦٧٨٩"

# حرکت ها

In [7]:
harekats="".join(harekat["sign"].values.tolist())

# یکسان سازی حروف

In [8]:
chardict={'ؤ':'و',
         'ي':'ی',
         'أ':'ا',
         'إ':'ا',
         'آ':'ا',
         'ك':'ک',
         'ة':'ه'}

# جمع مکسر

In [9]:
mokasar_dict=mokasar.set_index('jam').T.to_dict("list")

# حذف پسوند

In [10]:
pronouns = [
    "ام",
    "ات",
    "اش",
    "مان",
    "تان",
    "شان"
]


 سرزمینهایمان:
 سرزمین + های + مان

ضمیر‌های متصل آخر را حذف می‌کنیم

تر* و ها* را حذف می‌کنیم

ی هکسره را حذف می کنیم

In [11]:
def norm_suffix(t):
    for pr in pronouns:
        if t.endswith("\u200c"+pr):
            t = t.replace("\u200c"+pr, "")
            break
    if "\u200c"+"ها" in t:
            t = t.replace(t[t.rfind("\u200c"):], "")
    if "\u200c"+"تر" in t:
        t = t.replace(t[t.rfind("\u200c"):], "")
    if t.endswith("\u200c"+"ی"):
        t = t.replace("\u200c"+"ی", "")
    if t.endswith("\u200c"+"ای"):
        t = t.replace("\u200c"+"ای", "")
    return t

# حذف پیشوند

In [12]:
prefix = ["بیش", "پس", "پی", "بی","به"]

In [13]:
def norm_prefix(t):
    for pr in prefix:
        if t.startswith(pr+"\u200c"):
            t = t.replace(pr+"\u200c", "")
            break
    return t

# ریشه یابی افعال

In [14]:
prons=[
    "م",
    "ی",
    "یم",
    "ید",
    "ند"
]

In [15]:
def get_all_past(verb):
    verbs={}
    all_verbs=[verb]
    for pr in prons:
        verbs[verb+pr]=verb
        all_verbs.append(verb+pr)
    for v in all_verbs:
        verbs["می"+v]=verb
        verbs["می\u200c"+v]=verb
    return verbs

In [16]:
def get_all_present(verb):
    verbs={}
    all_verbs=[verb]
    for pr in prons:
        verbs[verb+pr]=verb
        all_verbs.append(verb+pr)
    verbs[verb+"د"]=verb
    all_verbs.append(verb+"د")
    for v in all_verbs:
        verbs["می"+v]=verb
        verbs["می\u200c"+v]=verb
        verbs["ب"+v]=verb
    return verbs

In [17]:
verbs_dict={}
for past in verbs["past"]:
    d=get_all_past(past)
    verbs_dict.update(d)
for present in verbs["present"]:
    d=get_all_present(present)
    verbs_dict.update(d)

# کلمات چند جزیی

بعد از اینکه از نیم‌فاصله استفاده کردیم و کلمات را جدا کردیم حال می‌توانیم نیم فاصله را حذف کنیم تا کلمات چند جزیی یکسان شوند

In [18]:
def hspace(t):
    t2=t
    if "\u200c" in t:
        t2=t.replace("\u200c", "")
    return t2

# Normalization

In [19]:
def normalize_doc(doc):
    normed=[]
    doc=set(doc)
    for t in doc:
        if t in stop_words:
            t=""
            continue
        t=t.translate(str.maketrans('','',chars_to_remove)) #علائم نگارشی اعداد فارسی انگلیسی عربی و...
        t=t.translate(str.maketrans('','',harekats)) #حرکت های زبان فارسی
        t=t.translate(str.maketrans(chardict)) # یکسان سازی حروف
        if t in mokasar_dict.keys(): # جمع مکسر
            t=mokasar_dict[t][0]
        t=norm_suffix(t) # حذف پسوند
        t=norm_prefix(t)
        if t in verbs_dict.keys(): # ریشه یابی فعل
            t=verbs_dict[t]
        t=hspace(t)
        normed.append(t)
    return normed

In [20]:
%%time
normalized_tokens=[]
for i in tokens:
    normalized_tokens.append(normalize_doc(i))

Wall time: 38.7 s


# build inverted index

In [21]:
def build_inverted_index(tokens):
    inv_idx=OrderedDict()
    for docid in range(len(tokens)):
        for token in tokens[docid]:
            if token in inv_idx.keys():
                inv_idx[token].append(docid)
            else:
                inv_idx[token]=[docid]
    return inv_idx

In [22]:
%%time
inv_idx=build_inverted_index(normalized_tokens)

Wall time: 808 ms


In [23]:
def single_query(q):
    if q[0] not in inv_idx.keys():
        return "no occurrences found"
    postings = inv_idx[q[0]]
    return data.loc[data.index[postings]]["url"]

In [24]:
def multi_query(q):
    postings=[]
    ranks={}
    for w in q:
        if w in inv_idx.keys():
            for doc_id in inv_idx[w]:
                if doc_id in ranks.keys():
                    ranks[doc_id]+=1
                else:
                    ranks[doc_id]=1
                    
    ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)
    postings=[idx for idx,rank in ranks]
    return data.loc[data.index[postings]]["url"],ranks[0][1]

In [46]:
quer="علی‌الحساب"
q=normalize_doc(quer.split())
single_query(q)

4667    https://www.isna.ir/news/98041507605/مبالغ-اضا...
5148    https://www.isna.ir/news/98091712625/برداشت-۱۰...
6990    https://www.isna.ir/news/98091712625/برداشت-۱۰...
6993    https://www.isna.ir/news/98091712625/برداشت-۱۰...
Name: url, dtype: object

In [63]:
quer="یزدی‌زاده"
q=normalize_doc(quer.split())
print(q)
single_query(q)

['یزدیزاده']


930    https://www.isna.ir/news/99111914509/رحمان-رضا...
Name: url, dtype: object

In [65]:
quer="علی‌الحساب"
q=normalize_doc(quer.split())
print(q)
single_query(q)

['علیالحساب']


4667    https://www.isna.ir/news/98041507605/مبالغ-اضا...
5148    https://www.isna.ir/news/98091712625/برداشت-۱۰...
6990    https://www.isna.ir/news/98091712625/برداشت-۱۰...
6993    https://www.isna.ir/news/98091712625/برداشت-۱۰...
Name: url, dtype: object

In [66]:
quer="عصاره فضائل ملت"
q=normalize_doc(quer.split())
print(q)
multi_query(q)

['عصاره', 'ملت', 'فضائل']


(3155    https://www.isna.ir/news/98112216539/فروزنده-ا...
 6726    https://www.isna.ir/news/98050200866/واکنش-این...
 6729    https://www.isna.ir/news/98050200866/واکنش-این...
 1133    https://www.isna.ir/news/98030401704/کولاکوویچ...
 1737    https://www.isna.ir/news/99011105670/بیانیه-جه...
                               ...                        
 6477    https://www.isna.ir/news/99121309975/اقدامات-ف...
 6515    https://www.isna.ir/news/99122619749/بهره-بردا...
 6516    https://www.isna.ir/news/99122721186/پیام-روسا...
 6523    https://www.isna.ir/news/99122721186/پیام-روسا...
 6989    https://www.isna.ir/news/98091611102/هرخانه-یک...
 Name: url, Length: 222, dtype: object,
 2)

In [67]:
quer="پاکت پریدنتال میکروب"
q=normalize_doc(quer.split())
print(q)
multi_query(q)

['میکروب', 'پریدنتال', 'پاکت']


(6995    https://www.isna.ir/news/98092015395/اصلی-ترین...
 6998    https://www.isna.ir/news/98092015395/اصلی-ترین...
 5929    https://www.isna.ir/news/99060504370/هشدارهایی...
 4040    https://www.isna.ir/news/99092821555/عرضه-شوین...
 4570    https://www.isna.ir/news/98031305827/خیز-منسوج...
 4648    https://www.isna.ir/news/98040904454/عرضه-نانو...
 5257    https://www.isna.ir/news/98102720683/معرفی-میک...
 5504    https://www.isna.ir/news/99012715875/بهترین-رف...
 5784    https://www.isna.ir/news/99042518891/حفظ-بهداش...
 6097    https://www.isna.ir/news/99072317682/علائم-تنف...
 6284    https://www.isna.ir/news/99092821555/عرضه-شوین...
 6386    https://www.isna.ir/news/99111309406/واکسن-روس...
 6489    https://www.isna.ir/news/99121712749/اختراع-نس...
 6491    https://www.isna.ir/news/99121712854/توصیه-وزا...
 6702    https://www.isna.ir/news/98041708893/شناسایی-ب...
 6708    https://www.isna.ir/news/98041708893/شناسایی-ب...
 6942    https://www.isna.ir/news/98082012554/آیا-کاپیت.

In [68]:
quer="بنیاد نخبگان و جهاد دانشگاهی"
q=normalize_doc(quer.split())
print(q)
multi_query(q)

['دانشگاهی', 'و', 'نخبگان', 'بنیاد', 'جهاد']


(4228    https://www.isna.ir/news/99120100312/رویداد-خا...
 4424    https://www.isna.ir/news/98021407530/تشریح-پیا...
 5135    https://www.isna.ir/news/98091208462/با-سیاسی-...
 1251    https://www.isna.ir/news/98050904866/تعهدات-جد...
 2756    https://www.isna.ir/news/98050301495/از-بررسی-...
                               ...                        
 6992    https://www.isna.ir/news/98091813677/تاکید-مسئ...
 6993    https://www.isna.ir/news/98091712625/برداشت-۱۰...
 6994    https://www.isna.ir/news/98091813677/تاکید-مسئ...
 6996    https://www.isna.ir/news/98092015327/انواع-دیا...
 6999    https://www.isna.ir/news/98092015327/انواع-دیا...
 Name: url, Length: 6889, dtype: object,
 6)

In [70]:
quer="سازمان حمایت از مصرف کننده"
q=normalize_doc(quer.split())
print(q)
multi_query(q)

['حمایت', 'سازمان', 'مصرف', 'کننده', 'از']


(5312    https://www.isna.ir/news/98111812880/باید-با-س...
 1808    https://www.isna.ir/news/99022316996/متن-کامل-...
 6665    https://www.isna.ir/news/98032008951/از-تیراند...
 6667    https://www.isna.ir/news/98032008951/از-تیراند...
 6669    https://www.isna.ir/news/98032008951/از-تیراند...
                               ...                        
 6992    https://www.isna.ir/news/98091813677/تاکید-مسئ...
 6994    https://www.isna.ir/news/98091813677/تاکید-مسئ...
 6996    https://www.isna.ir/news/98092015327/انواع-دیا...
 6997    https://www.isna.ir/news/98092216157/پنجمین-مو...
 6999    https://www.isna.ir/news/98092015327/انواع-دیا...
 Name: url, Length: 6566, dtype: object,
 8)

In [40]:
rank

5

In [28]:
len(inv_idx.keys())

34617

# یک ایده برای انکه میدان تبدیل به دان نشود

از آنجا که برای ریشه یابی افعال یک مپ درست کردیم دیگر پیشوند می را حذف نمیکنیم بلکه تمام شکل‌های فعل که می‌توان از بن ماضی و مضارع تولید کرد را می‌سازیم و سپس بررسی می‌کنیم که کل کلمه با افعال ما برابر است یا نه

درمورد بقیه‌ی پسوندها و پیشوندها نیز از طریق وجود نیم فاصله می‌توان فهمید که واقعا کلمه پسوند دارد یا نه

ایده‌ی دیگر در مورد پسوندهایی که با نیم‌فاصله وارد نمی‌شوند آن است که بعد از حذف پسوند بررسی کنیم طول کلمه بیشتر از ۳ باشد( برای کلماتی مثل طاها)


و ایده‌ی آخر آن است که بعد از تشکیل ایندکس پسوندها را جدا کنیم و بررسی کنیم با حذف پسوند کلمه‌ی به وجود آمده در دیکشنری وجود دارد یا خیر
البته می‌توانیم از یک دیکشنری کامل هم کمک بگیریم